## Populate the hospital prescribing DB

This (hacky) notebook is currently designed to be run from Google colab

We need to edit the code below to specify the details of the Postgres DB where we will put the data

In [ ]:
import psycopg2
import os
import pandas as pd

# Establish a connection to the PostgreSQL database

def get_connection():
  conn = psycopg2.connect(
    dbname="hospitalprescribing2",
    user="hospitals",
    password="",
    host="",
    port=""
  )
  return conn

Also, we need to get the dmd - unfortunately you need a (free) TRUD account for this:
https://isd.digital.nhs.uk/trud/

Modify the code below to use your download link for DMD from TRUD

In [ ]:
!wget https://isd.digital.nhs.uk/download/api/v1/keys/[YOUR KEY HERE]/content/items/24/nhsbsa_dmd_11.2.0_20231120000001.zip -O dmd.zip
!unzip dmd.zip

In [ ]:
import requests
import json
import os

# Step 1: Download the JSON from the provided URL
url = "https://opendata.nhsbsa.net/api/3/action/package_show?id=secondary-care-medicines-data-indicative-price"
response = requests.get(url)

# Ensure the request was successful
if response.status_code != 200:
    raise Exception("Failed to download the JSON.")

data = response.json()

# Step 2: Extract all the CSV file URLs from the JSON
csv_urls = [resource['url'] for resource in data['result']['resources'] if resource['format'] == 'CSV']

# Directory to save downloaded CSV files
save_dir = "downloaded_csvs"
os.makedirs(save_dir, exist_ok=True)

# Step 3: Download each CSV file
for csv_url in csv_urls:
    response = requests.get(csv_url)
    filename = os.path.join(save_dir, csv_url.split("/")[-1])

    with open(filename, 'wb') as file:
        file.write(response.content)

    print(f"Downloaded: {filename}")

print("All files downloaded successfully!")

Downloaded: downloaded_csvs/scmd_final_201904.csv
Downloaded: downloaded_csvs/scmd_final_201905.csv
Downloaded: downloaded_csvs/scmd_final_201906.csv
Downloaded: downloaded_csvs/scmd_final_201907.csv
Downloaded: downloaded_csvs/scmd_final_201908.csv
Downloaded: downloaded_csvs/scmd_final_201909.csv
Downloaded: downloaded_csvs/scmd_final_201910.csv
Downloaded: downloaded_csvs/scmd_final_201911.csv
Downloaded: downloaded_csvs/scmd_final_201912.csv
Downloaded: downloaded_csvs/scmd_final_202001.csv
Downloaded: downloaded_csvs/scmd_final_202002.csv
Downloaded: downloaded_csvs/scmd_final_202003.csv
Downloaded: downloaded_csvs/scmd_final_202004.csv
Downloaded: downloaded_csvs/scmd_final_202005.csv
Downloaded: downloaded_csvs/scmd_final_202006.csv
Downloaded: downloaded_csvs/scmd_final_202007.csv
Downloaded: downloaded_csvs/scmd_final_202008.csv
Downloaded: downloaded_csvs/scmd_final_202009.csv
Downloaded: downloaded_csvs/scmd_final_202010.csv
Downloaded: downloaded_csvs/scmd_final_202011.csv


In [ ]:


conn = get_connection()
cursor = conn.cursor()

# Create the temporary table
create_temp_table_query = """
CREATE TEMP TABLE temp_secondary_care_medicines_data (
    YEAR_MONTH INT,
    ODS_CODE TEXT,
    VMP_SNOMED_CODE BIGINT,
    VMP_PRODUCT_NAME TEXT,
    UNIT_OF_MEASURE_IDENTIFIER BIGINT,
    UNIT_OF_MEASURE_NAME TEXT,
    TOTAL_QUANITY_IN_VMP_UNIT FLOAT,
    INDICATIVE_COST FLOAT
)
"""
cursor.execute(create_temp_table_query)

# Create the mapping tables
create_vmp_mapping_table_query = """
CREATE TABLE IF NOT EXISTS vmp_code_name_mapping (
    VMP_SNOMED_CODE BIGINT,
    VMP_PRODUCT_NAME TEXT,
    PRIMARY KEY (VMP_SNOMED_CODE, VMP_PRODUCT_NAME)
)
"""
cursor.execute(create_vmp_mapping_table_query)

create_unit_mapping_table_query = """
CREATE TABLE IF NOT EXISTS unit_code_name_mapping (
    UNIT_OF_MEASURE_IDENTIFIER BIGINT,
    UNIT_OF_MEASURE_NAME TEXT,
    PRIMARY KEY (UNIT_OF_MEASURE_IDENTIFIER, UNIT_OF_MEASURE_NAME)
)
"""
cursor.execute(create_unit_mapping_table_query)

# Modify the main table
create_main_table_query = """
CREATE TABLE IF NOT EXISTS secondary_care_medicines_data (
    YEAR_MONTH INT,
    ODS_CODE TEXT,
    VMP_SNOMED_CODE BIGINT,
    UNIT_OF_MEASURE_IDENTIFIER BIGINT,
    TOTAL_QUANITY_IN_VMP_UNIT FLOAT,
    INDICATIVE_COST FLOAT
)
"""
cursor.execute(create_main_table_query)

csv_dir = "downloaded_csvs"
tsv_dir = "converted_tsvs"
if not os.path.exists(tsv_dir):
    os.makedirs(tsv_dir)

for filename in os.listdir(csv_dir):
    print(filename)
    if filename.endswith(".csv"):
        csv_filepath = os.path.join(csv_dir, filename)
        tsv_filepath = os.path.join(tsv_dir, filename.replace('.csv', '.tsv'))

        # Convert CSV to TSV with pandas
        df = pd.read_csv(csv_filepath)
        df.to_csv(tsv_filepath, sep='\t', index=False)

        with open(tsv_filepath, 'r') as file:
            next(file)  # Skip header row
            cursor.copy_from(file, 'temp_secondary_care_medicines_data', sep='\t', null="")

        # Transfer data to the mapping tables (ignoring duplicates)
        cursor.execute("""
        INSERT INTO vmp_code_name_mapping (VMP_SNOMED_CODE, VMP_PRODUCT_NAME)
        SELECT DISTINCT VMP_SNOMED_CODE, VMP_PRODUCT_NAME FROM temp_secondary_care_medicines_data
         ON CONFLICT (VMP_SNOMED_CODE, VMP_PRODUCT_NAME) DO NOTHING;

        """)

        cursor.execute("""
        INSERT INTO unit_code_name_mapping (UNIT_OF_MEASURE_IDENTIFIER, UNIT_OF_MEASURE_NAME)
        SELECT DISTINCT UNIT_OF_MEASURE_IDENTIFIER, UNIT_OF_MEASURE_NAME FROM temp_secondary_care_medicines_data

        ON CONFLICT (UNIT_OF_MEASURE_IDENTIFIER, UNIT_OF_MEASURE_NAME) DO NOTHING;

        """)

        # Transfer data to the main table
        cursor.execute("""
        INSERT INTO secondary_care_medicines_data (YEAR_MONTH, ODS_CODE, VMP_SNOMED_CODE, UNIT_OF_MEASURE_IDENTIFIER, TOTAL_QUANITY_IN_VMP_UNIT, INDICATIVE_COST)
        SELECT YEAR_MONTH, ODS_CODE, VMP_SNOMED_CODE, UNIT_OF_MEASURE_IDENTIFIER, TOTAL_QUANITY_IN_VMP_UNIT, INDICATIVE_COST
        FROM temp_secondary_care_medicines_data;
        """)

        # Clear the temporary table for the next iteration
        cursor.execute("TRUNCATE temp_secondary_care_medicines_data;")

conn.commit()
cursor.close()
conn.close()

"""
Now we also need
CREATE INDEX secondary_care_medicines_data_year_month_idx ON public.secondary_care_medicines_data (year_month);
CREATE INDEX secondary_care_medicines_data_vmp_snomed_code_idx ON public.secondary_care_medicines_data (vmp_snomed_code);

"""

print("Data has been successfully imported into the PostgreSQL table!")

scmd_final_202002.csv
scmd_final_201904.csv
scmd_final_202006.csv
scmd_final_202110.csv
scmd_final_202008.csv
scmd_provisional_202307.csv
scmd_final_201908.csv
scmd_final_202105.csv
scmd_wip_202211.csv
scmd_provisional_202304.csv
scmd_wip_202206.csv
scmd_final_201909.csv
scmd_wip_202205.csv
scmd_final_202011.csv
scmd_final_201906.csv
scmd_provisional_202303.csv
scmd_final_202111.csv
scmd_final_202109.csv
scmd_final_202202.csv
scmd_final_202001.csv
scmd_wip_202212.csv
scmd_wip_202208.csv
scmd_wip_202210.csv
scmd_final_201911.csv
scmd_provisional_202302.csv
scmd_final_202101.csv
scmd_final_202203.csv
scmd_final_202112.csv
scmd_final_202102.csv
scmd_final_202108.csv
scmd_provisional_202308.csv
scmd_final_201912.csv
scmd_wip_202209.csv
scmd_provisional_202309.csv
scmd_final_202010.csv
scmd_provisional_202305.csv
scmd_provisional_202301.csv
scmd_final_202012.csv
scmd_final_202201.csv
scmd_final_202005.csv
scmd_provisional_202306.csv
scmd_wip_202204.csv
scmd_final_201910.csv
scmd_wip_202207.

In [ ]:
!wget https://files.digital.nhs.uk/assets/ods/current/etr.zip

--2023-11-25 16:31:49--  https://files.digital.nhs.uk/assets/ods/current/etr.zip
Resolving files.digital.nhs.uk (files.digital.nhs.uk)... 52.84.52.98, 52.84.52.19, 52.84.52.76, ...
Connecting to files.digital.nhs.uk (files.digital.nhs.uk)|52.84.52.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206157 (201K) [application/zip]
Saving to: ‘etr.zip’

etr.zip             100%[===================>] 201.33K   793KB/s    in 0.3s    

2023-11-25 16:31:52 (793 KB/s) - ‘etr.zip’ saved [206157/206157]



In [ ]:
!unzip etr.zip

Archive:  etr.zip
  inflating: etr.csv                 
  inflating: etr.pdf                 


In [ ]:
import pandas as pd
import psycopg2
import os

# Step 1: Convert "file.csv" to "file.tsv"
csv_filepath = "etr.csv"
tsv_filepath = csv_filepath.replace('.csv', '.tsv')

df = pd.read_csv(csv_filepath,  header=None)

df = df.iloc[:, :2]
df.to_csv(tsv_filepath, sep='\t', index=False)


# Step 2: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 3: Create a new table for the data
create_table_query = """
CREATE TABLE IF NOT EXISTS ods_data (
    Code TEXT,
    Name TEXT
)
"""
cursor.execute(create_table_query)

# Step 4: Upload data from the TSV file into the new table
with open(tsv_filepath, 'r') as file:
    next(file)  # Skip header row
    cursor.copy_from(file, 'ods_data', sep='\t', null="")

conn.commit()
cursor.close()
conn.close()

print("Data has been successfully imported into the PostgreSQL table!")


Data has been successfully imported into the PostgreSQL table!


In [ ]:
import psycopg2
import xml.etree.ElementTree as ET

# Assuming the XML data file is named "ingredient_data.xml"
xml_filepath = "f_ingredient2_3140923.xml"
tsv_filepath = "ingredient_data.tsv"

# Step 1: Read and parse the XML data file
tree = ET.parse(xml_filepath)
root = tree.getroot()

# Step 2: Extract relevant data fields from the XML file and write to TSV
with open(tsv_filepath, 'w') as tsv_file:
    for ing in root.findall('ING'):
        isid = ing.find('ISID').text
        isiddt = ing.find('ISIDDT').text if ing.find('ISIDDT') is not None else '\\N'  # Using PostgreSQL's default representation for NULL
        isidprev = ing.find('ISIDPREV').text if ing.find('ISIDPREV') is not None else '\\N'
        invalid = ing.find('INVALID').text if ing.find('INVALID') is not None else '\\N'
        nm = ing.find('NM').text
        tsv_file.write(f"{isid}\t{isiddt}\t{isidprev}\t{invalid}\t{nm}\n")

# Step 3: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 4: Create a new table for the data (if not exists)
create_table_query = """
CREATE TABLE IF NOT EXISTS ingredient_data (
    ISID BIGINT,
    ISIDDT DATE,
    ISIDPREV BIGINT,
    INVALID INTEGER,
    NM TEXT
)
"""
cursor.execute(create_table_query)

# Step 5: Upload data from the TSV file into the new table
with open(tsv_filepath, 'r') as tsv_file:
    cursor.copy_from(tsv_file, 'ingredient_data', sep='\t', null="\\N")  # Using PostgreSQL's default representation for NULL

conn.commit()
cursor.close()
conn.close()

print("Ingredient data has been successfully imported into the PostgreSQL table!")


In [ ]:
import psycopg2
import xml.etree.ElementTree as ET
import glob
# Assuming the XML data file is named "ingredient_data.xml"

xml_filepath = "f_ingredient2_*.xml"
xml_filepath = glob.glob(xml_filepath)[0]


tsv_filepath = "ingredient_data.tsv"

# Step 1: Read and parse the XML data file
tree = ET.parse(xml_filepath)
root = tree.getroot()

# Step 2: Extract relevant data fields from the XML file and write to TSV
with open(tsv_filepath, 'w') as tsv_file:
    for ing in root.findall('ING'):
        isid = ing.find('ISID').text
        isiddt = ing.find('ISIDDT').text if ing.find('ISIDDT') is not None else '\\N'  # Using PostgreSQL's default representation for NULL
        isidprev = ing.find('ISIDPREV').text if ing.find('ISIDPREV') is not None else '\\N'
        invalid = ing.find('INVALID').text if ing.find('INVALID') is not None else '\\N'
        nm = ing.find('NM').text
        tsv_file.write(f"{isid}\t{isiddt}\t{isidprev}\t{invalid}\t{nm}\n")

# Step 3: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 4: Create a new table for the data (if not exists)
create_table_query = """
CREATE TABLE IF NOT EXISTS ingredient_data (
    ISID BIGINT,
    ISIDDT DATE,
    ISIDPREV BIGINT,
    INVALID INTEGER,
    NM TEXT
)
"""
cursor.execute(create_table_query)

# Step 5: Upload data from the TSV file into the new table
with open(tsv_filepath, 'r') as tsv_file:
    cursor.copy_from(tsv_file, 'ingredient_data', sep='\t', null="\\N")  # Using PostgreSQL's default representation for NULL

conn.commit()
cursor.close()
conn.close()

print("Ingredient data has been successfully imported into the PostgreSQL table!")


Ingredient data has been successfully imported into the PostgreSQL table!


In [ ]:
import psycopg2
import xml.etree.ElementTree as ET
import glob
# Assuming the XML data file is named "ingredient_data.xml"

xml_filepath = "f_vtm2_*.xml"
xml_filepath = glob.glob(xml_filepath)[0]
tsv_filepath = "vtm_data.tsv"

# Step 1: Read and parse the XML data file
tree = ET.parse(xml_filepath)
root = tree.getroot()

# Step 2: Extract relevant data fields from the XML file and write to TSV
with open(tsv_filepath, 'w') as tsv_file:
    for vtm in root.findall('VTM'):
        vtmid = vtm.find('VTMID').text
        invalid = vtm.find('INVALID').text if vtm.find('INVALID') is not None else '\\N'  # Using PostgreSQL's default representation for NULL
        nm = vtm.find('NM').text
        abbrevnm = vtm.find('ABBREVNM').text if vtm.find('ABBREVNM') is not None else '\\N'
        vtmidprev = vtm.find('VTMIDPREV').text if vtm.find('VTMIDPREV') is not None else '\\N'
        vtmiddt = vtm.find('VTMIDDT').text if vtm.find('VTMIDDT') is not None else '\\N'
        tsv_file.write(f"{vtmid}\t{invalid}\t{nm}\t{abbrevnm}\t{vtmidprev}\t{vtmiddt}\n")

# Step 3: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 4: Create a new table for the data (if not exists)
create_table_query = """
CREATE TABLE IF NOT EXISTS vtm_data (
    VTMID BIGINT,
    INVALID INTEGER,
    NM TEXT,
    ABBREVNM TEXT,
    VTMIDPREV BIGINT,
    VTMIDDT DATE
)
"""
cursor.execute(create_table_query)

# Step 5: Upload data from the TSV file into the new table
with open(tsv_filepath, 'r') as tsv_file:
    cursor.copy_from(tsv_file, 'vtm_data', sep='\t', null="\\N")  # Using PostgreSQL's default representation for NULL

conn.commit()
cursor.close()
conn.close()

print("VTM data has been successfully imported into the PostgreSQL table!")


VTM data has been successfully imported into the PostgreSQL table!


In [ ]:
import psycopg2
import xml.etree.ElementTree as ET

xml_filepath = "f_vmp2_*.xml"
xml_filepath = glob.glob(xml_filepath)[0]
tsv_filepath = "vmp_vtm_links.tsv"

# Step 1: Read and parse the XML data file
tree = ET.parse(xml_filepath)
root = tree.getroot()

# Step 2: Extract relevant data fields from the XML file and write to TSV
with open(tsv_filepath, 'w') as tsv_file:
    for vmp in root.find('VMPS').findall('VMP'):
        vpid = vmp.find('VPID').text
        vpid_prev = vmp.find('VPIDPREV').text if vmp.find('VPIDPREV') is not None else '\\N'
        vtmid = vmp.find('VTMID').text if vmp.find('VTMID') is not None else '\\N'  # Using PostgreSQL's default representation for NULL
        udfs = vmp.find('UDFS').text if vmp.find('UDFS') is not None else '\\N'
        udfs_uomcd = vmp.find('UDFS_UOMCD').text if vmp.find('UDFS_UOMCD') is not None else '\\N'
        unit_dose_uomcd = vmp.find('UNIT_DOSE_UOMCD').text if vmp.find('UNIT_DOSE_UOMCD') is not None else '\\N'
        tsv_file.write(f"{vpid}\t{vpid_prev}\t{vtmid}\t{udfs}\t{udfs_uomcd}\t{unit_dose_uomcd}\n")

# Step 3: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 4: Create a new table for the data (if not exists)
create_table_query = """
CREATE TABLE IF NOT EXISTS vmp_stuff (
    VPID BIGINT,
    VPID_PREV BIGINT,
    VTMID BIGINT,
    UDFS FLOAT,
    UDFS_UOMCD BIGINT,
    UNIT_DOSE_UOMCD BIGINT
)
"""
cursor.execute(create_table_query)

# Step 5: Upload data from the TSV file into the new table
with open(tsv_filepath, 'r') as tsv_file:
    cursor.copy_from(tsv_file, 'vmp_stuff', sep='\t', null="\\N")  # Using PostgreSQL's default representation for NULL

conn.commit()
cursor.close()
conn.close()

print("VMP to VTM links with dose information have been successfully imported into the PostgreSQL table!")


VMP to VTM links with dose information have been successfully imported into the PostgreSQL table!


In [ ]:
import psycopg2
import xml.etree.ElementTree as ET

# Use the provided XML data file
xml_filepath = "f_vmp2_*.xml"
xml_filepath = glob.glob(xml_filepath)[0]
tsv_filepath = "virtual_product_ingredient.tsv"

# Step 1: Read and parse the XML data file
tree = ET.parse(xml_filepath)
root = tree.getroot()

# Step 2: Extract relevant data fields from the XML file and write to TSV
with open(tsv_filepath, 'w') as tsv_file:
    for vpi in root.find('VIRTUAL_PRODUCT_INGREDIENT').findall('VPI'):
        vpid = vpi.find('VPID').text
        isid = vpi.find('ISID').text
        basis_strntcd = vpi.find('BASIS_STRNTCD').text if vpi.find('BASIS_STRNTCD') is not None else '\\N'
        strnt_nmrtr_val = vpi.find('STRNT_NMRTR_VAL').text if vpi.find('STRNT_NMRTR_VAL') is not None else '\\N'
        strnt_nmrtr_uomcd = vpi.find('STRNT_NMRTR_UOMCD').text if vpi.find('STRNT_NMRTR_UOMCD') is not None else '\\N'
        strnt_dnmtr_val = vpi.find('STRNT_DNMTR_VAL').text if vpi.find('STRNT_DNMTR_VAL') is not None else '\\N'
        strnt_dnmtr_uomcd = vpi.find('STRNT_DNMTR_UOMCD').text if vpi.find('STRNT_DNMTR_UOMCD') is not None else '\\N'
        tsv_file.write(f"{vpid}\t{isid}\t{basis_strntcd}\t{strnt_nmrtr_val}\t{strnt_nmrtr_uomcd}\t{strnt_dnmtr_val}\t{strnt_dnmtr_uomcd}\n")

# Step 3: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 4: Create a new table for the data (if not exists)
create_table_query = """
CREATE TABLE IF NOT EXISTS virtual_product_ingredient (
    VPID BIGINT,
    ISID BIGINT,
    BASIS_STRNTCD INTEGER,
    STRNT_NMRTR_VAL FLOAT,
    STRNT_NMRTR_UOMCD BIGINT,
    STRNT_DNMTR_VAL FLOAT,
    STRNT_DNMTR_UOMCD BIGINT
)
"""
cursor.execute(create_table_query)

# Step 5: Upload data from the TSV file into the new table
with open(tsv_filepath, 'r') as tsv_file:
    cursor.copy_from(tsv_file, 'virtual_product_ingredient', sep='\t', null="\\N")  # Using PostgreSQL's default representation for NULL

conn.commit()
cursor.close()
conn.close()

print("Virtual product ingredient data, including denominators, have been successfully imported into the PostgreSQL table!")


Virtual product ingredient data, including denominators, have been successfully imported into the PostgreSQL table!


In [ ]:
import psycopg2

sql = """CREATE TABLE unit_data (
	uom_cd int8 NULL,
	original_unit_name varchar NULL,
	basic_unit varchar NULL,
	number_of_basic_unit numeric NULL
);
"""

sql2 = """INSERT INTO public.unit_data (uom_cd,original_unit_name,basic_unit,number_of_basic_unit) VALUES
	 (258718000,'mmol','mmol',1),
	 (258773002,'ml','ml',1),
	 (767525000,'unit','unit',1),
	 (418931004,'GBq','GBq',1),
	 (229034000,'Mbq','Mbq',1),
	 (258719008,'umol','umol',1),
	 (258997004,'IU','IU',1),
	 (258731005,'ppm','ppm',1),
	 (258774008,'ul','ml',0.001),
	 (282113003,'nl','ml',0.000001);
INSERT INTO public.unit_data (uom_cd,original_unit_name,basic_unit,number_of_basic_unit) VALUES
	 (258684004,'mg','g',0.001),
	 (258685003,'ug','g',0.000001),
	 (258686002,'ng','g',0.000000001),
	 (258682000,'g','g',1);
"""
conn = get_connection()
cursor = conn.cursor()

cursor.execute(sql)

conn.commit()
cursor.close()


In [ ]:
cursor = conn.cursor()

cursor.execute(sql2)

conn.commit()
cursor.close()


In [ ]:
conn = get_connection()
sql = """
CREATE INDEX ingredient_data_isid_idx ON public.ingredient_data USING btree (isid);

CREATE INDEX vtm_data_vtmid_idx ON public.vtm_data USING btree (vtmid);

CREATE INDEX virtual_product_ingredient_isid_idx ON public.virtual_product_ingredient USING btree (isid);

CREATE INDEX virtual_product_ingredient_vpid_idx ON public.virtual_product_ingredient USING btree (vpid);
"""

cursor = conn.cursor()

cursor.execute(sql)

conn.commit()
cursor.close()


In [ ]:

conn = get_connection()
sql = """
CREATE INDEX secondary_care_medicines_data_vmp_snomed_code_idx ON public.secondary_care_medicines_data USING btree (vmp_snomed_code);

CREATE INDEX secondary_care_medicines_data_year_month_idx ON public.secondary_care_medicines_data USING btree (year_month);
"""

cursor = conn.cursor()

cursor.execute(sql)

conn.commit()
cursor.close()


In [ ]:
conn = get_connection()

# SQL commands to alter the table and update the new column
sql = """
ALTER TABLE ingredient_data
ADD COLUMN has_usage BOOLEAN DEFAULT FALSE;

UPDATE ingredient_data
SET has_usage = EXISTS (
    SELECT 1
    FROM secondary_care_medicines_data
    INNER JOIN virtual_product_ingredient
        ON virtual_product_ingredient.vpid = secondary_care_medicines_data.VMP_SNOMED_CODE
    WHERE virtual_product_ingredient.isid = ingredient_data.isid
);
"""

cursor = conn.cursor()

# Executing the SQL commands
cursor.execute(sql)

# Committing the changes to the database
conn.commit()

# Closing the cursor
cursor.close()

In [ ]:
import glob
import xml.etree.ElementTree as ET
conn =get_connection()
cursor = conn.cursor()
xml_filepath = "f_vmp2_*.xml"
xml_filepath = glob.glob(xml_filepath)[0]

route_tsv_filepath = "drug_route_data.tsv"

# Step 1: Read and parse the XML data file
tree = ET.parse(xml_filepath)
root = tree.getroot()


# Step 2.1: Extract Drug Route data fields and write to another TSV
with open(route_tsv_filepath, 'w') as tsv_file:
    for droute in root.findall(".//DROUTE"):
        vpid = droute.find('VPID').text if droute.find('VPID') is not None else '\\N'
        route_cd = droute.find('ROUTECD').text if droute.find('ROUTECD') is not None else '\\N'
        tsv_file.write(f"{vpid}\t{route_cd}\n")

# Step 3: Connect to the PostgreSQL database
conn = get_connection()
cursor = conn.cursor()

# Step 4: Create a new table for VMP data (if not exists)
# ... existing code to create vmp_stuff table ...

# Step 4.1: Create a new table for Drug Route data (if not exists)
create_route_table_query = """
CREATE TABLE IF NOT EXISTS drug_route (
    VPID BIGINT,
    ROUTECD BIGINT
)
"""
cursor.execute(create_route_table_query)

# Step 5: Upload VMP data from the TSV file into the new table
# ... existing code to upload VMP data ...

# Step 5.1: Upload Drug Route data from the TSV file into the new table
with open(route_tsv_filepath, 'r') as tsv_file:
    cursor.copy_from(tsv_file, 'drug_route', sep='\t', null="\\N")  # Using PostgreSQL's default representation for NULL

conn.commit()
cursor.close()
conn.close()

print("Data has been successfully imported into the PostgreSQL table!")

In [ ]:


def update_and_clean_drug_route_table():
    # SQL query to update the table
    update_query = """
    UPDATE drug_route
    SET ROUTECD = -1
    WHERE VPID IN (
        SELECT VPID
        FROM drug_route
        GROUP BY VPID
        HAVING COUNT(ROUTECD) > 1
    );
    """

    # SQL query to delete duplicates
    delete_duplicates_query = """
    DELETE FROM drug_route
    WHERE ctid NOT IN (
        SELECT MIN(ctid)
        FROM drug_route
        GROUP BY VPID
    );
    """
    try:
        # Establish a database connection
        conn = get_connection()
        cursor = conn.cursor()

        # Execute the update query
        cursor.execute(update_query)

        # Execute the delete duplicates query
        cursor.execute(delete_duplicates_query)

        # Commit the changes
        conn.commit()

        print("Table updated and duplicates removed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Call the function to update and clean the table
update_and_clean_drug_route_table()